In [1]:
import face_recognition
import argparse
import pickle
import cv2

In [18]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("--encodings", required=True,help="path to serialized db of facial encodings")
ap.add_argument("--image", required=True,help="path to input image")
ap.add_argument("--detection-method", type=str, default="cnn",help="face detection model to use: either `hog` or `cnn`")
args = vars(ap.parse_args(["--encodings","/Users/thejakamahaulpatha/PycharmProjects/Image Recognition/Project - Jurassic Park/encodings.pickle","--image","/Users/thejakamahaulpatha/PycharmProjects/Image Recognition/Project - Jurassic Park/examples/example2.jpeg"]))


In [19]:
# load the known faces and embeddings
print("[INFO] loading encodings...")
data = pickle.loads(open(args["encodings"], "rb").read())

# load the input image and convert it from BGR to RGB
image = cv2.imread(args["image"])
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


[INFO] loading encodings...


In [20]:
# detect the (x, y)-coordinates of the bounding boxes corresponding 
# to each face in the input image, then compute the facial embeddings for each face

print("[INFO] recognizing faces...")
boxes = face_recognition.face_locations(rgb,model=args["detection_method"])
encodings = face_recognition.face_encodings(rgb, boxes)

# initialize the list of names for each face detected
names = []

[INFO] recognizing faces...


In [21]:
# loop over the facial embeddings
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(data["encodings"],encoding)
    name = "Unknown"
    
    if True in matches:
        
		# find the indexes of all matched faces then initialize a
		# dictionary to count the total number of times each face was matched
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
		# loop over the matched indexes and maintain a count for
		# each recognized face face
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1
		# determine the recognized face with the largest number of
		# votes (note: in the event of an unlikely tie Python will
		# select first entry in the dictionary)
        name = max(counts, key=counts.get)
	
	# update the list of names
    names.append(name)

In [22]:
names

['Unknown', 'ian_malcolm', 'ellie_sattler', 'alan_grant', 'john_hammond']

In [23]:
matchedIdxs

[37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 152,
 183,
 192]

In [24]:
counts

{'john_hammond': 58, 'alan_grant': 3}

In [ ]:
# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes, names):
	# draw the predicted face name on the image
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)

# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)